# 0. 준비

## 1. 각종 필요 기능 import

In [5]:
# 데이터 처리 관련 준비
import pandas as pd
import numpy as np

# 크롤링 관련 준비
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# 대기를 위한 WebDriverWait import
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC # 기다리는 조건을 뭘로 할 지 정해줌

# 값을 기다릴 때마다 WebDriverWait을 호출해야하므로 함수로 정의해버리자.
def wait_until(xpath_str):
    WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH, xpath_str)))

## 2. 전처리 된 데이터 불러온 후, 검색 컬럼 생성

In [66]:
df = pd.read_csv('D:/DataSet/서울시_일반음식점_인허가_정보/gangnam_general_restaurant.csv', encoding = 'utf-8')
df.drop(columns = '번호', inplace = True)
df

,사업장명,소재지전화,소재지전체주소,도로명전체주소,업태구분명,좌표정보(x),좌표정보(y),Unnamed: 47
0,바다드림,NaN,서울특별시 강동구 성내동 427-47,"서울특별시 강동구 풍성로54길 33, 1층 (성내동)",횟집,211730.041860,447312.111904,NaN
1,청년피자 고덕상일점,NaN,서울특별시 강동구 고덕동 193-4,"서울특별시 강동구 상일로15길 22, 1층 (고덕동)",경양식,214946.249742,450680.438962,NaN
2,빰빠라,NaN,서울특별시 강동구 길동 388-3,"서울특별시 강동구 양재대로111길 18, 2층 (길동)",라이브카페,212172.882863,448356.930270,NaN
3,오우,NaN,서울특별시 중구 주교동 289-1,서울특별시 중구 을지로27길 32-16 (주교동),기타,199910.827092,451776.715598,NaN
4,금덕푸드,NaN,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",기타,200613.510298,451817.515367,NaN
...,...,...,...,...,...,...,...,...
124665,온기정,053 255 7949,서울특별시 강남구 삼성동 159-7 현대백화점,"서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)",일식,205210.358779,445154.422252,NaN
124666,놀부 공유주방 선릉역,NaN,서울특별시 강남구 대치동 896-28,"서울특별시 강남구 테헤란로68길 25, 지하1층 지하1호 (대치동)",한식,204603.136055,444606.621961,NaN
124667,인생갈비탕,NaN,서울특별시 강남구 역삼동 707-34 한신인터밸리24빌딩,"서울특별시 강남구 테헤란로 322, 한신인터밸리24빌딩 지하2층 B130호 (역삼동)",한식,204044.789949,444529.840427,NaN
124668,꾸어가게생선구이화곡점,0264089282,서울특별시 강서구 화곡동 827-2 1층,"서울특별시 강서구 등촌로13다길 35, 1층 (화곡동)",한식,187499.286366,448069.583478,NaN


In [50]:
df2 = pd.read_csv('D:/DataSet/서울시_일반음식점_인허가_정보/gangnam_general_restaurant.csv', encoding = 'utf-8')
df2.drop(columns = '번호', inplace = True)
df2

,사업장명,소재지전화,소재지전체주소,도로명전체주소,업태구분명,좌표정보(x),좌표정보(y),Unnamed: 47
0,바다드림,NaN,서울특별시 강동구 성내동 427-47,"서울특별시 강동구 풍성로54길 33, 1층 (성내동)",횟집,211730.041860,447312.111904,NaN
1,청년피자 고덕상일점,NaN,서울특별시 강동구 고덕동 193-4,"서울특별시 강동구 상일로15길 22, 1층 (고덕동)",경양식,214946.249742,450680.438962,NaN
2,빰빠라,NaN,서울특별시 강동구 길동 388-3,"서울특별시 강동구 양재대로111길 18, 2층 (길동)",라이브카페,212172.882863,448356.930270,NaN
3,오우,NaN,서울특별시 중구 주교동 289-1,서울특별시 중구 을지로27길 32-16 (주교동),기타,199910.827092,451776.715598,NaN
4,금덕푸드,NaN,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",기타,200613.510298,451817.515367,NaN
...,...,...,...,...,...,...,...,...
124665,온기정,053 255 7949,서울특별시 강남구 삼성동 159-7 현대백화점,"서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)",일식,205210.358779,445154.422252,NaN
124666,놀부 공유주방 선릉역,NaN,서울특별시 강남구 대치동 896-28,"서울특별시 강남구 테헤란로68길 25, 지하1층 지하1호 (대치동)",한식,204603.136055,444606.621961,NaN
124667,인생갈비탕,NaN,서울특별시 강남구 역삼동 707-34 한신인터밸리24빌딩,"서울특별시 강남구 테헤란로 322, 한신인터밸리24빌딩 지하2층 B130호 (역삼동)",한식,204044.789949,444529.840427,NaN
124668,꾸어가게생선구이화곡점,0264089282,서울특별시 강서구 화곡동 827-2 1층,"서울특별시 강서구 등촌로13다길 35, 1층 (화곡동)",한식,187499.286366,448069.583478,NaN


In [67]:
df['naver_keyword'] = '강남%20' + df['사업장명']
df

,사업장명,소재지전화,소재지전체주소,도로명전체주소,업태구분명,좌표정보(x),좌표정보(y),Unnamed: 47,naver_keyword
0,바다드림,NaN,서울특별시 강동구 성내동 427-47,"서울특별시 강동구 풍성로54길 33, 1층 (성내동)",횟집,211730.041860,447312.111904,NaN,강남%20바다드림
1,청년피자 고덕상일점,NaN,서울특별시 강동구 고덕동 193-4,"서울특별시 강동구 상일로15길 22, 1층 (고덕동)",경양식,214946.249742,450680.438962,NaN,강남%20청년피자 고덕상일점
2,빰빠라,NaN,서울특별시 강동구 길동 388-3,"서울특별시 강동구 양재대로111길 18, 2층 (길동)",라이브카페,212172.882863,448356.930270,NaN,강남%20빰빠라
3,오우,NaN,서울특별시 중구 주교동 289-1,서울특별시 중구 을지로27길 32-16 (주교동),기타,199910.827092,451776.715598,NaN,강남%20오우
4,금덕푸드,NaN,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",기타,200613.510298,451817.515367,NaN,강남%20금덕푸드
...,...,...,...,...,...,...,...,...,...
124665,온기정,053 255 7949,서울특별시 강남구 삼성동 159-7 현대백화점,"서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)",일식,205210.358779,445154.422252,NaN,강남%20온기정
124666,놀부 공유주방 선릉역,NaN,서울특별시 강남구 대치동 896-28,"서울특별시 강남구 테헤란로68길 25, 지하1층 지하1호 (대치동)",한식,204603.136055,444606.621961,NaN,강남%20놀부 공유주방 선릉역
124667,인생갈비탕,NaN,서울특별시 강남구 역삼동 707-34 한신인터밸리24빌딩,"서울특별시 강남구 테헤란로 322, 한신인터밸리24빌딩 지하2층 B130호 (역삼동)",한식,204044.789949,444529.840427,NaN,강남%20인생갈비탕
124668,꾸어가게생선구이화곡점,0264089282,서울특별시 강서구 화곡동 827-2 1층,"서울특별시 강서구 등촌로13다길 35, 1층 (화곡동)",한식,187499.286366,448069.583478,NaN,강남%20꾸어가게생선구이화곡점


In [68]:
df = df.head()
df

,사업장명,소재지전화,소재지전체주소,도로명전체주소,업태구분명,좌표정보(x),좌표정보(y),Unnamed: 47,naver_keyword
0,바다드림,NaN,서울특별시 강동구 성내동 427-47,"서울특별시 강동구 풍성로54길 33, 1층 (성내동)",횟집,211730.041860,447312.111904,NaN,강남%20바다드림
1,청년피자 고덕상일점,NaN,서울특별시 강동구 고덕동 193-4,"서울특별시 강동구 상일로15길 22, 1층 (고덕동)",경양식,214946.249742,450680.438962,NaN,강남%20청년피자 고덕상일점
2,빰빠라,NaN,서울특별시 강동구 길동 388-3,"서울특별시 강동구 양재대로111길 18, 2층 (길동)",라이브카페,212172.882863,448356.930270,NaN,강남%20빰빠라
3,오우,NaN,서울특별시 중구 주교동 289-1,서울특별시 중구 을지로27길 32-16 (주교동),기타,199910.827092,451776.715598,NaN,강남%20오우
4,금덕푸드,NaN,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",기타,200613.510298,451817.515367,NaN,강남%20금덕푸드


# 1. 크롤링

In [97]:
# 데이터를 담을 데이터 프레임 준비
c_columns = ['name','addresse','cat1','cat2','review_num','score', 'review']
data_frame = pd.DataFrame(columns = c_columns)

# 크롬 창 열기
browser = webdriver.Chrome(ChromeDriverManager().install()) # 크롬드라이버 자동 설치

# 별점과 메뉴, 리뷰 끌어오기
for i, keyword in enumerate(df['naver_keyword'].tolist()):

    print(keyword)
    
    # naver_keyword 순서대로 불러와서 검색
    naver_map_search_url = f"https://map.naver.com/v5/search/{keyword}"
    browser.get(naver_map_search_url)
    time.sleep(3)

    # 검색결과가 카테고리 + 리스트로 나올 때
    try:
        browser.switch_to.default_content()
        iframe_1 = browser.find_element(By.XPATH, '/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[1]/combined-search-list/salt-search-list/nm-external-frame-bridge/nm-iframe/iframe')
        browser.switch_to.frame(iframe_1)
        restaurant = browser.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div[1]/ul/li[1]/div[1]/a')
        restaurant.click()
        time.sleep(4)

        # 리스트의 첫 번째 요소
        browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/a/div[1]/div/span[1]').click()
        
        time.sleep(3)
        #사업장명
        name =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[1]').text
        # 사업장 주소
        address = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/ul/li[2]/div/a/span[1]').text
        # 음식 종류
        cat1 =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text

        # 별점이 존재한다면..?
        try:
            # 별점 추가
            score = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
        except:
            # 없으면 결측값
            score = np.nan

        #메뉴 버튼 클릭
        browser.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]/span').click()
        time.sleep(3)

        # 메뉴
        cat2 = []
        menus = browser.find_elements(By.CLASS_NAME, "list_link")
        for menu in menus:
            cat2.append(menu.text)
        time.sleep(3)

    except:
        # 검색결과가 리스트로 나올 때
        try:
            browser.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/a/div[1]/div/span[1]').click()
            time.sleep(3)
            #사업장명
            name =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[1]').text
            # 사업장 주소
            address = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/ul/li[2]/div/a/span[1]').text
            # 음식 종류
            cat1 =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text
            
            # 별점이 존재한다면..?
            try:
                # 별점 추가
                score = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
            except:
                # 없으면 결측값
                score = np.nan

            #메뉴 버튼 클릭
            browser.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]/span').click()
            time.sleep(3)

            # 메뉴
            cat2 = []
            menus = browser.find_elements(By.CLASS_NAME, "list_link")
            for menu in menus:
                cat2.append(menu.text)
            time.sleep(3)
        except:
            # 음식점 페이지가 한 번에 떴을 때
            try:
                #사업장명
                name =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[1]').text
                # 사업장 주소
                address = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/ul/li[2]/div/a/span[1]').text
                # 음식 종류
                cat1 =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text

                # 별점이 존재한다면..?
                try:
                    # 별점 추가
                    score = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
                except:
                    # 없으면 결측값
                    score = np.nan

                #메뉴 버튼 클릭
                browser.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]/span').click()
                time.sleep(3)

                # 메뉴
                cat2 = []
                menus = browser.find_elements(By.CLASS_NAME, "list_link")
                for menu in menus:
                    cat2.append(menu.text)
                time.sleep(3)

            except:
                name = np.nan

    print("데이터는 : ", name)

    
        



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\xilverh0ya\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18712\1149485146.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install()) # 크롬드라이버 자동 설치


강남%20바다드림
데이터는 :  nan
강남%20오우
데이터는 :  nan
강남%20금덕푸드
데이터는 :  nan


In [98]:
# 크롬 창 열기
browser = webdriver.Chrome(ChromeDriverManager().install()) # 크롬드라이버 자동 설치

# 별점과 메뉴, 리뷰 끌어오기
for i, keyword in enumerate(df['naver_keyword'].tolist()):

    print(keyword)
    
    # naver_keyword 순서대로 불러와서 검색
    naver_map_search_url = f"https://map.naver.com/v5/search/{keyword}"
    browser.get(naver_map_search_url)
    time.sleep(3)

    browser.switch_to.default_content()
    iframe_1 = browser.find_element(By.XPATH, '/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[1]/combined-search-list/salt-search-list/nm-external-frame-bridge/nm-iframe/iframe')
    browser.switch_to.frame(iframe_1)
    restaurant = browser.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div[1]/ul/li[1]/div[1]/a')
    restaurant.click()
    time.sleep(4)
    browser.switch_to.default_content()
    iframe_2 = browser.find_element(By.XPATH, '/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-place-bridge/div/nm-external-frame-bridge/nm-iframe/iframe')
    browser.switch_to.frame(iframe_2)
    restaurant_name = browser.find_element(By.XPATH, '//*[@id="_title"]/span[1]') #음식점 이름
    #/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/span[2]
    restautant_clss = browser.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[2]/div[1]/div[1]/span[2]') #음식점 분류
    time.sleep(2)

    print(restaurant_name, restautant_clss)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\xilverh0ya\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18712\1864824900.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install()) # 크롬드라이버 자동 설치


강남%20바다드림


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div[2]/div[1]/ul/li[1]/div[1]/a"}
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x0027B8F3+2406643]
	Ordinal0 [0x0020AF31+1945393]
	Ordinal0 [0x000FC748+837448]
	Ordinal0 [0x001292E0+1020640]
	Ordinal0 [0x0012957B+1021307]
	Ordinal0 [0x00156372+1205106]
	Ordinal0 [0x001442C4+1131204]
	Ordinal0 [0x00154682+1197698]
	Ordinal0 [0x00144096+1130646]
	Ordinal0 [0x0011E636+976438]
	Ordinal0 [0x0011F546+980294]
	GetHandleVerifier [0x004E9612+2498066]
	GetHandleVerifier [0x004DC920+2445600]
	GetHandleVerifier [0x00314F2A+579370]
	GetHandleVerifier [0x00313D36+574774]
	Ordinal0 [0x00211C0B+1973259]
	Ordinal0 [0x00216688+1992328]
	Ordinal0 [0x00216775+1992565]
	Ordinal0 [0x0021F8D1+2029777]
	BaseThreadInitThunk [0x7531FA29+25]
	RtlGetAppContainerNamedObjectPath [0x772E7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x772E7A4E+238]
	(No symbol) [0